In [1]:
using NBInclude, NLopt
@nbinclude("startup_FRA.ipynb")

C0 is Country1(300.0, 216.0, 432.0, 780.0, 0.0048, NaN, NaN, NaN, NaN, NaN, 0.0, [1.0, 0.0], 0.68, 0.6024096385542168, 0.23023448152535933, NaN, NaN, NaN)
Startup run in 31.646703004837036 secs.


# 1) Find the parameters of the benchmark model

We calibrate our model to determine the values of $\alpha$, $m$ and $q$ (and $\lambda$).

In [2]:
function distance(m1,m2)
    out = 0
    for i in 1:length(m1)
        out += (m1[i] - m2[i]) ^2
    end
    return(out)
end

## compute the objective to minimise
function objective_fr(α,m,q0,qa)
    newC = update_Country1(α=α, m=m, q0=q0, qa=qa) 

    ## simulate
    newV = find_Surplus(newC, reltol=1e-12, findΨ=false)
    unemp = find_Unemp(newV, newC, find_with=false, find_without=false) 
    moments_emp = moments_france(unemp.real, atol=1e-8)
    
  #  IJulia.clear_output(true)
  #  println("Current parameters: $α, $m, $q0, $qa")
  #  println("Simulated moments: $moments_emp")
    
    distance(moments_emp,moments_target)
end

## distance minimisation with NLopt
@time parmin = let
    opt = Opt(:LN_BOBYQA, 4)
    lower_bounds!(opt, [0.1 , 0., 0.001, -0.1])
    upper_bounds!(opt, [10. , 1., 0.2, 0.1 ])
    xtol_rel!(opt,1e-12)
    min_objective!(opt, (v,g)-> objective_fr(v[1],v[2],v[3],v[4]))
    
    init =  [0.6, 0.7, 0.01, 0.] 
    
    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    minx
end
;

Param found with convergence SUCCESS, the value of the objective is 0.00016341599713460718
Parameters: 
 - α= 0.24099219070226158
 - m= 0.27615709934433336
 - q0= 0.06830434097107665
 - qa= -7.159841894302883e-5
 11.283769 seconds (49.49 M allocations: 4.098 GiB, 5.01% gc time, 66.55% compilation time: <1% of which was recompilation)


# 2) Sensitivity: find the parameters with alternative values of $q_0$
Find the parameters for $q_0=0.005, 0.01, 0.025, 0.05, 0.1$.

In [3]:
function find_france(q0)
    opt = Opt(:LN_BOBYQA, 3)
    lower_bounds!(opt, [0.1 , 1e-6, -0.1])
    upper_bounds!(opt, [20. , 10., 0.1])
    xtol_rel!(opt,1e-12)
    min_objective!(opt, (v,g)-> objective_fr(v[1],v[2],q0, v[3]))
    
    init =  [0.6, 0.7, 0.] 
    
    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - qa= ", minx[3])
    return(minx, minf)
end

q0_vec = [0.01 0.025 0.05 0.075 0.1]
#[0.005 0.01 0.025 0.05 0.1]
amqa_vec = zeros(3,5)
objectivevalue = zeros(5)
for (i,q0) in enumerate(q0_vec)
    @time amqa_vec[:,i], objectivevalue[i] =  find_france(q0)
end

println(objectivevalue)
out = vcat(q0_vec,amqa_vec)
show(out)
out

Param found with convergence SUCCESS, the value of the objective is 0.0010136077147515567
Parameters: 
 - α= 0.22849509692057998
 - m= 0.04073389331497086
 - qa= -3.712082211914822e-5
 21.990824 seconds (304.12 M allocations: 28.671 GiB, 18.65% gc time, 0.53% compilation time)
Param found with convergence SUCCESS, the value of the objective is 0.0005578103075928695
Parameters: 
 - α= 0.3117478108428948
 - m= 0.10482000306832813
 - qa= -4.8412393275282786e-5
  4.271295 seconds (63.71 M allocations: 5.983 GiB, 18.20% gc time)
Param found with convergence SUCCESS, the value of the objective is 0.00022721201801000255
Parameters: 
 - α= 0.2607359798656053
 - m= 0.20474363913214377
 - qa= -6.099046958730697e-5
  0.445127 seconds (5.26 M allocations: 466.068 MiB, 7.65% gc time)
Param found with convergence SUCCESS, the value of the objective is 0.0001538553522629591
Parameters: 
 - α= 0.23967448574973638
 - m= 0.3029013451226111
 - qa= -7.60204363645744e-5
  0.558719 seconds (6.77 M allocatio

4×5 Matrix{Float64}:
  0.01         0.025        0.05         0.075        0.1
  0.228495     0.311748     0.260736     0.239674     0.407114
  0.0407339    0.10482      0.204744     0.302901     0.442741
 -3.71208e-5  -4.84124e-5  -6.09905e-5  -7.60204e-5  -9.68599e-5

# 3) Sensitivity: find the parameters with alternative values of $\xi$

In [4]:
function objective_fr(α,m,q0,qa,ξ)
    newC = update_Country1(α=α, m=m, q0=q0, qa=qa, ξ=ξ) 

    ## simulate
    newV = find_Surplus(newC, reltol=1e-12)
    unemp = find_Unemp(newV, newC, find_with=false, find_without=false) 
    moments_emp = moments_france(unemp.real, atol=1e-8)
    distance(moments_emp,moments_target)
end

function find_france(ξ)
    opt = Opt(:LN_BOBYQA, 4)
    lower_bounds!(opt, [0.1 , 0., 0.001, -0.1])
    upper_bounds!(opt, [20. , 10., 0.2, 0.1 ])
    xtol_rel!(opt,1e-12)
    min_objective!(opt, (v,g)-> objective_fr(v[1],v[2],v[3],v[4],ξ))
    
    init =  [0.6, 0.7, 0.01, 0.] 
    
    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    return (minx,minf)
end


ξ_vec = [0 0.25 0.5 0.75 1]
amq_vec = zeros(4,5)
objectivevalue = zeros(5)
for (i,ξ) in enumerate(ξ_vec)
    @time amq_vec[:,i], objectivevalue[i] =  find_france(ξ)
end


println(objectivevalue)
out = vcat(ξ_vec,amq_vec)
show(out)
out

Param found with convergence SUCCESS, the value of the objective is 0.00019404215193616556
Parameters: 
 - α= 0.23934241405757434
 - m= 0.2299570521500436
 - q0= 0.056983800301068
 - qa= -6.455667869386995e-5
  3.396047 seconds (31.20 M allocations: 2.659 GiB, 7.39% gc time, 9.26% compilation time)
Param found with convergence SUCCESS, the value of the objective is 0.00020398141773788307
Parameters: 
 - α= 0.2821534976596658
 - m= 0.27818244149516286
 - q0= 0.06732680444966312
 - qa= -8.148878990117824e-5
  3.348954 seconds (31.20 M allocations: 2.662 GiB, 7.09% gc time, 0.91% compilation time)
Param found with convergence SUCCESS, the value of the objective is 0.0002412000860313436
Parameters: 
 - α= 0.31618156437721234
 - m= 0.29151429088961134
 - q0= 0.06956356120250261
 - qa= -9.246745113898855e-5
  3.183337 seconds (27.24 M allocations: 2.310 GiB, 6.24% gc time)


┌ Warning: Instability detected. Aborting
└ @ SciMLBase ~/.julia/packages/SciMLBase/2HZ5m/src/integrator_interface.jl:602
┌ Warning: Instability detected. Aborting
└ @ SciMLBase ~/.julia/packages/SciMLBase/2HZ5m/src/integrator_interface.jl:602
┌ Warning: Instability detected. Aborting
└ @ SciMLBase ~/.julia/packages/SciMLBase/2HZ5m/src/integrator_interface.jl:602


Param found with convergence SUCCESS, the value of the objective is 0.00020549309582860604
Parameters: 
 - α= 0.33691609577306375
 - m= 0.38448245638360956
 - q0= 0.09094442237528229
 - qa= -0.0001170258767406586
  3.470029 seconds (30.62 M allocations: 2.598 GiB, 6.91% gc time, 6.00% compilation time: 87% of which was recompilation)
Param found with convergence SUCCESS, the value of the objective is 0.00024212571211023673
Parameters: 
 - α= 0.3602653271142106
 - m= 0.3810033261602324
 - q0= 0.08951757491483667
 - qa= -0.00012405218047344347
  1.593237 seconds (15.35 M allocations: 1.306 GiB, 7.22% gc time)
[0.00019404215193616556, 0.00020398141773788307, 0.0002412000860313436, 0.00020549309582860604, 0.00024212571211023673]
[0.0 0.25 0.5 0.75 1.0; 0.23934241405757434 0.2821534976596658 0.31618156437721234 0.33691609577306375 0.3602653271142106; 0.2299570521500436 0.27818244149516286 0.29151429088961134 0.38448245638360956 0.3810033261602324; 0.056983800301068 0.06732680444966312 0.069

5×5 Matrix{Float64}:
  0.0          0.25         0.5          0.75          1.0
  0.239342     0.282153     0.316182     0.336916      0.360265
  0.229957     0.278182     0.291514     0.384482      0.381003
  0.0569838    0.0673268    0.0695636    0.0909444     0.0895176
 -6.45567e-5  -8.14888e-5  -9.24675e-5  -0.000117026  -0.000124052